In [1]:
from nemo.collections.asr.models import EncDecMultiTaskModel

/home/tassilo/projects/OnnxNemoSTTInference/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[NeMo W 2025-12-22 21:11:39 nemo_logging:405] Megatron num_microbatches_calculator not found, using Apex version.
OneLogger: Setting error_handling_strategy to DISABLE_QUIETLY_AND_REPORT_METRIC_ERROR for rank (rank=0) with OneLogger disabled. To override: explicitly set error_handling_strategy parameter.
No exporters were provided. This means that no telemetry data will be collected.
[NeMo W 2025-12-22 21:11:43 nemo_logging:405] /home/tassilo/projects/OnnxNemoSTTInference/.venv/lib/python3.11/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
      warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWar

In [2]:
canary:EncDecMultiTaskModel = EncDecMultiTaskModel.from_pretrained(model_name="nvidia/canary-180m-flash", map_location="cpu").eval() # type: ignore

[NeMo I 2025-12-22 21:11:51 nemo_logging:393] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2025-12-22 21:11:51 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1152 tokens
[NeMo I 2025-12-22 21:11:51 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-12-22 21:11:51 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-12-22 21:11:51 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-12-22 21:11:51 nemo_logging:393] Tokenizer SentencePieceTokenizer initialized with 1024 tokens
[NeMo I 2025-12-22 21:11:51 nemo_logging:393] Aggregate vocab size: 5248


[NeMo W 2025-12-22 21:11:54 nemo_logging:405] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    use_lhotse: true
    input_cfg: null
    tarred_audio_filepaths: null
    manifest_filepath: null
    sample_rate: 16000
    shuffle: true
    num_workers: 8
    pin_memory: true
    prompt_format: canary2
    max_tps: 25
    max_duration: 40.0
    text_field: answer
    lang_field: target_lang
    use_bucketing: true
    bucket_duration_bins:
    - - 3.56
      - 30
    - - 3.56
      - 77
    - - 4.608
      - 38
    - - 4.608
      - 88
    - - 5.48
      - 49
    - - 5.48
      - 106
    - - 6.05
      - 52
    - - 6.05
      - 109
    - - 6.85
      - 54
    - - 6.85
      - 124
    - - 7.914
      - 59
    - - 7.914
      - 137
    - - 8.52
      - 67
    - - 8.52
      - 158
    - - 9.51
      - 67
    - - 9.51
      - 153
    - - 10.29
      -

[NeMo I 2025-12-22 21:11:54 nemo_logging:393] PADDING: 0


Error getting class at nemo.collections.asr.modules.transformer.get_nemo_transformer: Located non-class of type 'function' while loading 'nemo.collections.asr.modules.transformer.get_nemo_transformer'


[NeMo I 2025-12-22 21:12:01 nemo_logging:393] Model EncDecMultiTaskModel was successfully restored from /home/tassilo/.cache/huggingface/hub/models--nvidia--canary-180m-flash/snapshots/b12ab418510d093e83890178fd0e8b0d0f7918a6/canary-180m-flash.nemo.


In [ ]:
# canary.encoder.export("canary/encoder.onnx")

In [ ]:
from export import exportable_forward, stft
import types